# Домашнее задание №4
Группа: РИМ-150950

ФИО: Эрмиш Александр Александрович

## Задание
*Для изучения CSS селекторов https://flukeout.github.io/*

- Найдите сайт с открытыми таблицами (например, статистика IMDB, погодные данные, вакансии) и с помощью requests + BeautifulSoup извлеките несколько колонок и загрузите в DataFrame.

- Установите заголовок запроса (headers={'User-Agent': 'Mozilla/5.0 ...'}) и добавьте time.sleep(1) между запросами при парсинге нескольких страниц.

- Используйте soup.select() с CSS-селектором, чтобы выбрать интересующие элементы и извлечь текст или атрибуты.

- После получения данных с веба отметьте, как часто встречаются пропущенные значения или некорректные форматы (например, строка вместо числа)  и примените методы Pandas по очистке.

In [56]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from time import sleep
from urllib.parse import urljoin
import re

In [57]:
# URL для парсинга
url = 'https://www.imdb.com/chart/top/'

# Заголовки запроса для имитации реального браузера
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.9',
    'Accept-Encoding': 'gzip, deflate, br',
    'Referer': 'https://www.imdb.com/',
    'Connection': 'keep-alive',
}

## Парсинг веб-страницы

Теперь выполним парсинг страницы с использованием `requests` и `BeautifulSoup`, используя CSS-селекторы для извлечения нужной информации.

In [58]:
# Отправляем GET-запрос к странице
try:
    session = requests.Session()
    response = session.get(url, headers=headers)
    
    # Проверяем успешность запроса
    if response.status_code == 200:
        print("Страница успешно загружена")
    else:
        print(f"Ошибка при загрузке страницы: {response.status_code}")
        exit()
        
except Exception as e:
    print(f"Ошибка при выполнении запроса: {e}")
    exit()

# Создаем объект BeautifulSoup для парсинга HTML
soup = BeautifulSoup(response.content, 'html.parser')
# soup = BeautifulSoup(response.text, 'html.parser')


Страница успешно загружена


In [59]:
# Очищаем списки перед заполнением
titles = []
years = []
ratings = []
directors = []
actors_list = []

# Найдем ссылки на страницы фильмов с нужными данными
movie_containers = soup.select('li.ipc-metadata-list-summary-item')

print(f"Найдено фильмов: {len(movie_containers)}")

Найдено фильмов: 25


In [ ]:
containers_to_visit = movie_containers[:]

for i, container in enumerate(containers_to_visit):
    try:
        # Извлекаем данные с главной страницы
        title_element = container.select_one('h3.ipc-title__text')
        year_element = container.select('span.cli-title-metadata-item')[0] if container.select('span.cli-title-metadata-item') else None
        rating_element = container.select_one('span.ipc-rating-star--rating')
        link_element = container.select_one('a.ipc-title-link-wrapper')
        
        # Сохраняем базовые данные
        title = title_element.get_text().strip() if title_element else np.nan
        year = year_element.get_text().strip() if year_element else np.nan
        rating = rating_element.get_text().strip() if rating_element else np.nan
        
        # Получаем ссылку на страницу фильма
        if link_element and link_element.get('href'):
            relative_url = link_element.get('href')
            full_url = f'https://www.imdb.com{relative_url}'
            
            # Отправляем запрос на страницу фильма
            film_response = requests.get(full_url, headers=headers, timeout=10)
            
            if film_response.status_code == 200:
                film_soup = BeautifulSoup(film_response.content, 'html.parser')
                
                # Извлекаем режиссера
                director = np.nan
                director_section = film_soup.select_one('li[data-testid="title-pc-principal-credit"] span.ipc-metadata-list-item__label')
                if director_section and "Director" in director_section.get_text():
                    director_element = film_soup.select_one('li[data-testid="title-pc-principal-credit"] a.ipc-metadata-list-item__list-content-item')
                    if director_element:
                        director = director_element.get_text().strip()
                
                # Извлекаем актеров
                actors = []
                stars_section = film_soup.select_one('li[data-testid="title-pc-principal-credit"] a.ipc-metadata-list-item__label[href*="fullcredits"]')
                if stars_section and "Stars" in stars_section.get_text():
                    actor_elements = film_soup.select('li[data-testid="title-pc-principal-credit"] ul.ipc-inline-list li.ipc-inline-list__item a.ipc-metadata-list-item__list-content-item')
                    for actor_element in actor_elements[:3]:  # Берем первых 3 актеров
                        actor_name = actor_element.get_text().strip()
                        actors.append(actor_name)
                
                actors_str = ', '.join(actors) if actors else np.nan
                
                # Добавляем данные в списки
                titles.append(title)
                years.append(year)
                ratings.append(rating)
                directors.append(director)
                actors_list.append(actors_str)
            else:
                print(f"  Ошибка загрузки страницы фильма: {film_response.status_code}")
                titles.append(title)
                years.append(year)
                ratings.append(rating)
                directors.append(np.nan)
                actors_list.append(np.nan)
        else:
            print(f"  Не найдена ссылка для фильма: {title}")
            titles.append(title)
            years.append(year)
            ratings.append(rating)
            directors.append(np.nan)
            actors_list.append(np.nan)
        
        # Добавляем задержку между запросами
        sleep(2)
        
    except Exception as e:
        print(f"  Ошибка при обработке фильма {i+1}: {e}")
        titles.append(np.nan)
        years.append(np.nan)
        ratings.append(np.nan)
        directors.append(np.nan)
        actors_list.append(np.nan)
        sleep(2)

# Создаем DataFrame извлеченных данных
df = pd.DataFrame({
    'Title': titles,
    'Year': years,
    'Rating': ratings,
    'Director': directors,
    'Actors': actors_list
})

In [61]:
# Выводим DataFrame
df

,Title,Year,Rating,Director,Actors
0,The Shawshank Redemption,1994,9.3,Frank Darabont,"Frank Darabont, Stephen King, Frank Darabont"
1,The Godfather,1972,9.2,Francis Ford Coppola,"Francis Ford Coppola, Mario Puzo, Francis Ford..."
2,The Dark Knight,2008,9.1,Christopher Nolan,NaN
3,The Godfather Part II,1974,9.0,Francis Ford Coppola,"Francis Ford Coppola, Francis Ford Coppola, Ma..."
4,12 Angry Men,1957,9.0,Sidney Lumet,"Sidney Lumet, Reginald Rose, Henry Fonda"
5,The Lord of the Rings: The Return of the King,2003,9.0,Peter Jackson,NaN
6,Schindler's List,1993,9.0,Steven Spielberg,"Steven Spielberg, Thomas Keneally, Steven Zail..."
7,The Lord of the Rings: The Fellowship of the Ring,2001,8.9,Peter Jackson,NaN
8,Pulp Fiction,1994,8.8,Quentin Tarantino,"Quentin Tarantino, Quentin Tarantino, Roger Avary"
9,"The Good, the Bad and the Ugly",1966,8.8,Sergio Leone,NaN


In [62]:
# Сохраняем в CSV
df.to_csv('imdb_top_films.csv', index=False, encoding='utf-8-sig')
print("\nДанные успешно сохранены в файл 'imdb_top_films.csv'")


Данные успешно сохранены в файл 'imdb_top_films.csv'


## Очистка данных

In [63]:
# Выводим информацию о DataFrame
print("Информация о DataFrame:")
df.info()

# Проверяем количество пропущенных значений
print("\nКоличество пропущенных значений в каждом столбце:")
print(df.isnull().sum())

# Проверяем общее количество строк
print(f"\nОбщее количество строк: {len(df)}")

# Проверяем уникальные значения в столбцах
print(f"\nКоличество уникальных значений в каждом столбце:")
print(df.nunique())

Информация о DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Title     25 non-null     object
 1   Year      25 non-null     object
 2   Rating    25 non-null     object
 3   Director  25 non-null     object
 4   Actors    18 non-null     object
dtypes: object(5)
memory usage: 1.1+ KB

Количество пропущенных значений в каждом столбце:
Title       0
Year        0
Rating      0
Director    0
Actors      7
dtype: int64

Общее количество строк: 25

Количество уникальных значений в каждом столбце:
Title       25
Year        21
Rating       8
Director    17
Actors      17
dtype: int64


In [66]:
# Очистка данных

# Заменяем пропущенные значения в числовых столбцах средним значением
if df['Year'].dtype in ['int64', 'float64']:
    df['Year'].fillna(df['Year'].median(), inplace=True)

if df['Rating'].dtype in ['int64', 'float64']:
    df['Rating'].fillna(df['Rating'].mean(), inplace=True)

# Для текстовых столбцов заменяем пропущенные значения
df['Title'].fillna('Unknown Title', inplace=True)
df['Director'].fillna('Unknown Director', inplace=True)
df['Actors'].fillna('Unknown Actors', inplace=True)  # Добавляем обработку нового столбца

# Убираем лишние пробелы в текстовых столбцах
for col in ['Title', 'Director', 'Actors']:
    if col in df.columns:
        df[col] = df[col].astype(str).str.strip()
        df[col] = df[col].str.replace(r'\s+', ' ', regex=True)

# Проверяем типы данных и при необходимости корректируем их
df['Year'] = pd.to_numeric(df['Year'], errors='coerce').astype('Int64')  # Int64 позволяет NaN значения
df['Rating'] = pd.to_numeric(df['Rating'], errors='coerce')

# Выводим обновленную информацию о DataFrame
print("\nОбновленная информация о DataFrame после очистки:")
df.info()

# Проверяем количество пропущенных значений после очистки
print("\nКоличество пропущенных значений после очистки:")
print(df.isnull().sum())


Обновленная информация о DataFrame после очистки:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Title     25 non-null     object 
 1   Year      25 non-null     Int64  
 2   Rating    25 non-null     float64
 3   Director  25 non-null     object 
 4   Actors    25 non-null     object 
dtypes: Int64(1), float64(1), object(3)
memory usage: 1.1+ KB

Количество пропущенных значений после очистки:
Title       0
Year        0
Rating      0
Director    0
Actors      0
dtype: int64


C:\Users\sermi\AppData\Local\Temp\ipykernel_5304\1870891886.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Rating'].fillna(df['Rating'].mean(), inplace=True)
C:\Users\sermi\AppData\Local\Temp\ipykernel_5304\1870891886.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

Fo

In [67]:
df

,Title,Year,Rating,Director,Actors
0,The Shawshank Redemption,1994,9.3,Frank Darabont,"Frank Darabont, Stephen King, Frank Darabont"
1,The Godfather,1972,9.2,Francis Ford Coppola,"Francis Ford Coppola, Mario Puzo, Francis Ford..."
2,The Dark Knight,2008,9.1,Christopher Nolan,Unknown Actors
3,The Godfather Part II,1974,9.0,Francis Ford Coppola,"Francis Ford Coppola, Francis Ford Coppola, Ma..."
4,12 Angry Men,1957,9.0,Sidney Lumet,"Sidney Lumet, Reginald Rose, Henry Fonda"
5,The Lord of the Rings: The Return of the King,2003,9.0,Peter Jackson,Unknown Actors
6,Schindler's List,1993,9.0,Steven Spielberg,"Steven Spielberg, Thomas Keneally, Steven Zail..."
7,The Lord of the Rings: The Fellowship of the Ring,2001,8.9,Peter Jackson,Unknown Actors
8,Pulp Fiction,1994,8.8,Quentin Tarantino,"Quentin Tarantino, Quentin Tarantino, Roger Avary"
9,"The Good, the Bad and the Ugly",1966,8.8,Sergio Leone,Unknown Actors
